# Reading data from EPT

## Introduction

This tutorial describes how to use [Conda], [Entwine], [PDAL], and [GDAL] to
read data from the [USGS 3DEP AWS Public Dataset]. We will be using PDAL's
[readers.ept] to fetch data, we will filter it for noise using [filters.outlier],
we will classify the data as ground/not-ground using [filters.smrf], and we will
write out a digital terrain model with {ref}`writers.gdal`. Once our elevation model
is constructed, we will use GDAL [gdaldem] operations to create hillshade, slope,
and color relief.

## Write the Pipeline

PDAL uses the concept of [pipelines] to describe the reading, filtering, and writing
of point cloud data. We will construct a pipeline that will do a number of things
in succession.

:::{figure} images/pipeline-example-overview.png
:scale: 75%

Pipeline diagram. The data are read from the [Entwine Point Tile] resource at
<https://usgs.entwine.io> for Iowa using {ref}`readers.ept` and filtered through a
number of steps until processing is complete. The data are then written to
an `iowa.laz` and `iowa.tif` file.
:::

In [ ]:
import os
import sys

conda_env_path = os.environ.get('CONDA_PREFIX', sys.prefix)
proj_data = os.path.join(os.path.join(conda_env_path, 'share'), 'proj')
os.environ["PROJ_DATA"] = proj_data

In [ ]:
import pdal

In [ ]:
pipeline = pdal.Reader.ept("https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json",
                    bounds="([-10425171.940, -10423171.940], [5164494.710, 5166494.710])")
pipeline |= pdal.Filter.range(limits="Classification![7:7]")
pipeline |= pdal.Filter.assign(assignment="Classification[:]=0")
pipeline |= pdal.Filter.reprojection(out_srs="EPSG:26915")
pipeline |= pdal.Filter.smrf()
pipeline |= pdal.Filter.range(limits="Classification[2:2]")
pipeline.execute()
print(f"Processed point cloud contains {len(pipeline.arrays[0])} points")